In [1]:
import os 
import numpy as np
import pandas as pd 
from datetime import datetime as dt

In [2]:
work_dir = os.getcwd()

In [3]:
test_data = pd.read_csv(os.path.join(work_dir, "data", "output_weekly.csv"))
newDate1 = []
for i in range(len(test_data)):
    newDate1.append(dt.strptime(test_data['Date'][i], "%Y/%m/%d").strftime('%Y/%m/%d'))
test_data["Date"] = newDate1
test_data.head(5)

,Date,FPA,FPXI,IQDE,IQDY,CHIQ,GEM,ADRA,PAF,IEMG,...,MOTI,REMX,VEU,VXUS,AXJL,EMCG,XSOE,DNL,IHDG,DBAP
0,2016/01/25,0.019079,-0.003397,0.043354,0.031897,0.001017,0.052927,-0.005268,-0.065416,0.056029,...,0.048589,0.004020,0.039623,0.041447,0.030861,0.047829,0.032221,0.051002,0.039069,-0.045680
1,2016/02/01,-0.021951,-0.027957,-0.011482,-0.020219,-0.034659,-0.013858,-0.036860,-0.042535,-0.009889,...,0.000000,0.026839,-0.023511,-0.019094,-0.007118,-0.030631,-0.025282,-0.024040,-0.043373,-0.021488
2,2016/02/08,-0.002942,-0.026209,-0.010722,-0.020489,0.016269,-0.016048,-0.026039,0.006139,-0.010714,...,0.000000,-0.025145,-0.009755,-0.013210,-0.001763,-0.003098,-0.016556,-0.007752,-0.012870,0.003749
3,2016/02/16,0.009909,0.020188,0.016797,0.024859,0.025615,0.013661,0.014473,0.001161,0.009434,...,0.020073,0.031589,0.011580,0.011539,0.031126,0.000000,0.015368,0.018646,0.014780,0.023810
4,2016/02/22,-0.018036,-0.006542,-0.017429,-0.016402,-0.040816,-0.026072,-0.018279,0.000286,-0.028800,...,0.000000,0.045671,-0.012379,-0.010000,-0.025700,-0.035756,0.000000,-0.017773,-0.005332,-0.015504


In [4]:
USG1M = pd.read_csv(os.path.join(work_dir, "Risk_free_rate", "DGS1MO.csv"))

newDate = []
for i in range(len(USG1M)):
    date_str =  str(USG1M['Time'][i])[:4] + '-' + str(USG1M['Time'][i])[4:6] + '-' + str(USG1M['Time'][i])[6:]
    newDate.append(dt.strptime(date_str, '%Y-%m-%d').strftime('%Y/%m/%d'))
USG1M["Date"] = newDate

weekly_return = []
for i in range(len(USG1M)):
    try:
         weekly_return.append(float(USG1M["DGS1MO"][i])/52/7)
    except:
        weekly_return.append(0)
USG1M["DGS1MO"] = weekly_return
USG1M.head()

,Time,DGS1MO,Date
0,20140509,0.000055,2014/05/09
1,20140512,0.000055,2014/05/12
2,20140513,0.000082,2014/05/13
3,20140514,0.000055,2014/05/14
4,20140515,0.000027,2014/05/15


In [5]:
cum_ret = []
last_week_pos = 0
cur_week_pos = 0

for i in range(len(USG1M)):
    
    if(USG1M["Date"][i] in newDate1):
        if(last_week_pos == 0): #base case
            last_week_pos = i
            cur_week_pos = i
            cum_ret.append(np.nan)
        else:
            last_week_pos = cur_week_pos
            cur_week_pos = i
            cum_ret.append(sum(USG1M['DGS1MO'][last_week_pos:cur_week_pos]))
    else:
        cum_ret.append(np.nan)

USG1M['Monthly_Rf'] = cum_ret 
USG1M.head()
new_USG1M = USG1M.loc[:, ["Date","Monthly_Rf"]]

In [6]:
Results = test_data.merge(new_USG1M, on = "Date")
Results.tail(5)
Results.to_csv(os.path.join(work_dir, "data", "output_weekly_with_rf.csv"), index = False)